Данная часть кода отвечает за создание базы данных, на которой будет тренироваться, валидироваться и тестироваться модель. База данных преобразованна в протокол SpeakerDiarization pyannote.database. 

In [1]:
#открываю табличку с отобранными для датасета данными 
import pandas as pd

df = pd.read_csv('audios_diar.csv')

In [2]:
#всего в датасете 29 аннотированных аудиозаписей, 1053 минуты аудио, 921 минута аннтированных данных
df

,SessionID,Length,Length_of_eaf,Length_of_eaf_final
0,KK-Gibeon-20220813-1a-C,34:59:00,"1,5",2
1,KK-Gibeon-20220813-1b-C,35:10:00,"35,2",36
2,KK-Gibeon-20220813-1c-C,30:21:00,"29,8",30
3,KK-Gibeon-20220813-3a-C,44:14:00,"44,2",45
4,KK-Grootfontein-20201202-13-PSY,31:43:00,"31,7",32
5,KK-Sesfontein-19750000-1a-Garob,8:05:00,"5,9",6
6,KK-Hoachanas-20201204-17-PSY,23:44:00,"23,7",24
7,KK-Windhoek-2015-Pears-RG,3:43,"3,7",4
8,KK-Karibib-20201201-05-PSY,65:59:00,"65,9",66
9,KK-Karibib-20201201-06-PSY,38:00:00,"37,7",38


In [3]:
#преображаю столбец с названием файлов в список
list_of_wavs = df['SessionID'].tolist()

In [4]:
#разделяю данные на тренировочный, тестовый и валидационный датасеты, 80%/10%/10%
split_1 = int(0.8 * len(list_of_wavs))
split_2 = int(0.9 * len(list_of_wavs))
train_filenames = list_of_wavs[:split_1]
dev_filenames = list_of_wavs[split_1:split_2]
test_filenames = list_of_wavs[split_2:]

In [6]:
#сохраняю названия файлов в тестовый, валидационный и тренировочный файлы с расширением .lst
with open('train.lst', 'w') as f:
    for item in train_filenames:
        f.write("%s\n" % item)
   

In [7]:
with open('development.lst', 'w') as f2:
    for item in dev_filenames:
        f2.write("%s\n" % item)

In [8]:
with open('test.lst', 'w') as f3:
    for item in test_filenames:
        f3.write("%s\n" % item)

In [9]:
#импортирую pympi, бибилиотеку, которая работает с файлами формата .eaf, в которых сохранены наши аннотации
import pympi
import os

In [10]:
#функция, создающая список i, где каждое i: tuple(начало, окончание сегмена), лэйбел сегмента, название файла
def annotated_list(list_for_annotation):
    full_annot_list = []
    for filename in list_for_annotation:
        eaf = pympi.Elan.Eaf(str(filename + '.eaf'))
        reftiers = []
        list_of_ref = []
        for i in eaf.tiers:
            if 'ref' in i:
                reftiers.append(i)
        list_of_annot = []
        for i in reftiers:
            annot = eaf.get_annotation_data_for_tier(i)
            for j in annot:
                list_of_annot.append((j[0:2],i, filename))
                list_of_ref.append(i)
        for i in range(len(list_of_annot) - 1):
            min_idx = i
            for j in range(i + 1, len(list_of_annot)):
                if list_of_annot[min_idx][0] > list_of_annot[j][0]:
                    min_idx = j

            if i != min_idx:
                list_of_annot[i], list_of_annot[min_idx] = list_of_annot[min_idx], list_of_annot[i]
        full_annot_list.append(list_of_annot)
    return full_annot_list

In [11]:
# функция, преобразующая наш список в rttm файл
def write_rttm(full_annot_list):
    for i in full_annot_list:
        for j in i:
            fields = ['SPEAKER', j[2], '1', str(j[0][0] /  1000.0),str((j[0][1] - j[0][0])/ 1000.0),
              '<NA>', '<NA>', j[1], '<NA>', '<NA>']
            line = ' '.join(fields)
            f.write(line.encode('utf-8'))
            f.write(b'\n')

In [12]:
#делаем список из длительностей аннотацей аудиозаписей
list_of_length_for_uem = df['Length_of_eaf_final'].tolist()

In [13]:
#делим их также, как делили аудиозаписи
train_uem = list_of_length_for_uem[:split_1] 
dev_uem = list_of_length_for_uem[split_1:split_2] 
test_uem = list_of_length_for_uem[split_2:] 

In [14]:
#делаем список с длительностями аннотаций во всех датасетах 
uem_train = []
uem_dev = []
uem_test = []
for a in train_uem:
    uem_train.append(a-1)
for a in dev_uem:
    uem_dev.append(a-1)
for a in test_uem:
    uem_test.append(a-1)
    

In [15]:
#делаем список из сегментов, лейблов и названий файлов train 
annotaited_train = annotated_list(train_filenames)

In [16]:
#записываем этот список в .rttm
with open('train.rttm', 'wb') as f:
    write_rttm(annotaited_train)
        
        

In [17]:
#записываем продолжительности аннотаций в .uem 
with open('train.uem', 'wb') as f:
    for i in range(len(train_filenames)):
        fields = [train_filenames[i], 'NA', '0.000',str(uem_train[i] * 60) + '.000']
        line = ' '.join(fields)
        f.write(line.encode('utf-8'))
        f.write(b'\n')
    

In [18]:
#то же самое с test 
annotaited_test = annotated_list(test_filenames)

In [20]:
with open('test.rttm', 'wb') as f:
    write_rttm(annotaited_test)

In [21]:
with open('test.uem', 'wb') as f:
    for i in range(len(test_filenames)):
        fields = [test_filenames[i], 'NA', '0.000',str(uem_test[i] *60) + '.000']
        line = ' '.join(fields)
        f.write(line.encode('utf-8'))
        f.write(b'\n')
    

In [22]:
#то же самое с dev
annotaited_dev = annotated_list(dev_filenames)

In [23]:
with open('dev.rttm', 'wb') as f:
    write_rttm(annotaited_dev)

In [24]:
with open('dev.uem', 'wb') as f:
    for i in range(len(dev_filenames)):
        fields = [dev_filenames[i], 'NA', '0.000',str(uem_dev[i] * 60) + '.000']
        line = ' '.join(fields)
        f.write(line.encode('utf-8'))
        f.write(b'\n')
    